In [10]:
def prepare_train_features(examples,tokenizer,pad_on_right=True,max_length=512,doc_stride=128):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    
    tokenized_examples = tokenizer(
            examples["para"],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_length,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )
    print(tokenized_examples)
    
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")
    
    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    remove = []
    # print(tokenized_examples)
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        
        input_ids = tokenized_examples["input_ids"][i]
        
        cls_index = input_ids.index(tokenizer.eos_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
#         answer = examples["answers"][sample_index]
        answer = examples['target_text']
        # If no answers are given, set the cls_index as answer.
#         try:
        # Start/end character index of the answer in the text.
        context = examples['para']
        
        
        start_char = context.lower().find(answer.lower())
        if start_char == -1: # not find
#             tokenized_examples["start_positions"].append(cls_index)
#             tokenized_examples["end_positions"].append(cls_index)
            remove.append(i)
            continue
            
        
        end_char = start_char + len(answer)

        # Start token index of the current span in the text.
        token_start_index = 0
        
        while sequence_ids[token_start_index] != 0:
            token_start_index += 1

        # End token index of the current span in the text.
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] !=0:
            token_end_index -= 1
        
        print(token_start_index, token_end_index)
        # Detect if the answer is out of the span (in which case we remove this split).
        if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
            remove.append(i)
        
        else:
            # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
            # Note: we could go after the last offset if the answer is the last word (edge case).
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)
    new_input_ids = []
    new_target_ids = []
    print(tokenized_examples)
    for i,(ids, attn) in enumerate(zip(tokenized_examples['input_ids'],tokenized_examples['attention_mask'])):
        if i not in remove:
            new_input_ids.append(ids)
            new_target_ids.append(attn)
    tokenized_examples['input_ids'] = new_input_ids
    tokenized_examples['attention_mask']=new_target_ids
    return tokenized_examples

In [2]:
from transformers import RobertaTokenizerFast
from tqdm import tqdm as tqdm

In [3]:
!pip3 install ipywidgets

In [3]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [15]:
example = {"para": "Were Scott Derrickson and Ed Wood of the same nationality?</s> Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.</s>  He lives in Los Angeles, California.</s>  He is best known for directing horror films such as \"Sinister\", \"The Exorcism of Emily Rose\", and \"Deliver Us From Evil\", as well as the 2016 Marvel Cinematic Universe installment, \"Doctor Strange.\"", "para_label": 1, "sents_label": [1, -1, -1]}
example['target_text']='American director'
#example = {"para": "Which magazine was started first Arthur's Magazine or First for Women?</s> Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.</s>  Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.</s>  In May 1846 it was merged into \"Godey's Lady's Book\".", "para_label": 1, "sents_label": [1, -1, -1], "target_text": "Arthur's Magazine", "ans_label": 1}
#example = {'para': 'The Oberoi family is part of a hotel company that has a head office in what city?</s> The Ritz-Carlton Jakarta is a hotel and skyscraper in Jakarta, Indonesia and 14th Tallest building in Jakarta.</s>  It is located in city center of Jakarta, near Mega Kuningan, adjacent to the sister JW Marriott Hotel.</s>  It is operated by The Ritz-Carlton Hotel Company.</s>  The complex has two towers that comprises a hotel and the Airlangga Apartment respectively.</s>  The hotel was opened in 2005.', 'para_label': -1, 'sents_label': [-1, -1, -1, -1, -1], 'target_text': 'Delhi'}

In [17]:
res = prepare_train_features(example, tokenizer)

{'input_ids': [[0, 43822, 1699, 6113, 24214, 261, 8, 2344, 3132, 9, 5, 276, 26241, 116, 2, 1699, 6113, 24214, 261, 36, 5400, 550, 545, 6, 18069, 43, 16, 41, 470, 736, 6, 2441, 9408, 8, 3436, 4, 2, 1437, 91, 1074, 11, 1287, 1422, 6, 886, 4, 2, 1437, 91, 16, 275, 684, 13, 13062, 8444, 3541, 215, 25, 22, 42043, 4742, 1297, 22, 133, 3015, 37690, 1809, 9, 7770, 3199, 1297, 8, 22, 21502, 8538, 4890, 1740, 26780, 1297, 25, 157, 25, 5, 336, 8714, 23488, 5183, 15384, 18513, 6, 22, 41152, 18680, 72, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [22]:
c = 0
for i in res['attention_mask'][0]:
    if(i == 1):
        c += 1
    
print(c)

95


In [34]:
doc_stride = 128
max_length = 512
tokenized_examples = tokenizer(
            example["para"],
            truncation="only_second",
            max_length=max_length,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )
t = tokenizer(
    example["para"].split("</s> ")[0],
    truncation="only_second",
    max_length=max_length,
    stride=doc_stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)

In [36]:
print(tokenized_examples)
print(t)

{'input_ids': [[0, 43822, 1699, 6113, 24214, 261, 8, 2344, 3132, 9, 5, 276, 26241, 116, 2, 1699, 6113, 24214, 261, 36, 5400, 550, 545, 6, 18069, 43, 16, 41, 470, 736, 6, 2441, 9408, 8, 3436, 4, 2, 1437, 91, 1074, 11, 1287, 1422, 6, 886, 4, 2, 1437, 91, 16, 275, 684, 13, 13062, 8444, 3541, 215, 25, 22, 42043, 4742, 1297, 22, 133, 3015, 37690, 1809, 9, 7770, 3199, 1297, 8, 22, 21502, 8538, 4890, 1740, 26780, 1297, 25, 157, 25, 5, 336, 8714, 23488, 5183, 15384, 18513, 6, 22, 41152, 18680, 72, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [10]:
res
#answer label 

{'input_ids': [[0, 32251, 4320, 21, 554, 78, 9411, 18, 10202, 50, 1234, 13, 2691, 116, 2, 9411, 18, 10202, 36, 1366, 3305, 2383, 1366, 3761, 43, 21, 41, 470, 17205, 675, 3569, 1027, 11, 3378, 11, 5, 753, 212, 3220, 4, 2, 1437, 34525, 30, 255, 4, 104, 4, 9411, 6, 24, 3520, 173, 30, 20630, 83, 4, 29273, 6, 344, 4, 725, 4, 11996, 27092, 6, 4143, 3351, 102, 20810, 6, 1813, 272, 4, 37828, 6, 8, 643, 4, 2, 1437, 96, 392, 504, 3761, 24, 21, 21379, 88, 22, 534, 4636, 219, 18, 4645, 18, 5972, 845, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [14]:
answers = res['input_ids'][0][res['start_positions'][0]:res['end_positions'][0]+1]
answers_string = tokenizer.decode(answers)
answers_string

" Arthur's Magazine"

In [13]:
answers

[9411, 18, 10202]